# RunnableParallel

In [13]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [14]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

In [15]:
chat_template_books = ChatPromptTemplate.from_template(
    '''
    Suggest three of the best intermediate-level {programming language} books. 
    Answer only by listing the books.
    '''
)

chat_template_projects = ChatPromptTemplate.from_template(
    '''
    Suggest three interesting {programming language} projects suitable for intermediate-level programmers. 
    Answer only by listing the projects.
    '''
)

chat_template_time = ChatPromptTemplate.from_template(
    '''
    I'm an intermediate level programmer,

    Consider the following literature:
    {books}

    Also, consider the following projects:
    {projects}

    Roughly how much time would it take me to complete the literature and the projects?
    '''
)

In [16]:
chat = ChatOpenAI(model_name = 'gpt-4', 
                  model_kwargs = {'seed':365},
                  temperature = 0,
                  max_tokens = 500)

/opt/anaconda3/envs/langchain_env_benji/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3519: UserWarning: Parameters {'seed'} should be specified explicitly. Instead they were passed in as part of `model_kwargs` parameter.
  if await self.run_code(code, result, async_=asy):


In [17]:
string_parser = StrOutputParser()

In [18]:
chain_books = chat_template_books | chat | string_parser

chain_projects = chat_template_projects | chat | string_parser

In [19]:
chain_parallel = RunnableParallel({'books':chain_books, 'projects':chain_projects})

In [20]:
chain_parallel.invoke({'programming language':'Python'})

{'books': '1. "Fluent Python: Clear, Concise, and Effective Programming" by Luciano Ramalho\n2. "Python Cookbook: Recipes for Mastering Python 3" by David Beazley and Brian K. Jones\n3. "Effective Python: 90 Specific Ways to Write Better Python" by Brett Slatkin',
 'projects': '1. Building a Web Scraper using BeautifulSoup.\n2. Developing a simple Machine Learning model using Scikit-learn.\n3. Creating a basic web application using Django or Flask.'}

In [21]:
chain_parallel.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            

In [22]:
chain_time = (RunnableParallel({'books':chain_books, 'projects':chain_projects})
              | chat_template_time | chat |string_parser) 



In [23]:
chain_time.invoke({'programming language': 'Python'})

'The time it takes to complete the literature and the projects can vary greatly depending on several factors such as your current skill level, the amount of time you can dedicate each day, your reading speed, and how quickly you grasp new concepts. \n\nHowever, as a rough estimate:\n\n1. "Fluent Python: Clear, Concise, and Effective Programming" - This book is around 800 pages. If you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.\n\n2. "Python Cookbook: Recipes for Mastering Python 3" - This book is around 700 pages. Again, if you read and practice for about 2 hours a day, it might take you around 1-2 months to complete.\n\n3. "Effective Python: 90 Specific Ways to Write Better Python" - This book is around 230 pages. If you read and practice for about 2 hours a day, it might take you around 2-3 weeks to complete.\n\nAs for the projects:\n\n1. Building a Web Scraper using BeautifulSoup and Requests - If you work on it for about 2-3 hours a 

In [24]:
chain_time.get_graph().print_ascii()

            +-------------------------------+              
            | Parallel<books,projects>Input |              
            +-------------------------------+              
                   ***               ***                   
                ***                     ***                
              **                           **              
+--------------------+              +--------------------+ 
| ChatPromptTemplate |              | ChatPromptTemplate | 
+--------------------+              +--------------------+ 
           *                                   *           
           *                                   *           
           *                                   *           
    +------------+                      +------------+     
    | ChatOpenAI |                      | ChatOpenAI |     
    +------------+                      +------------+     
           *                                   *           
           *                            